In [1]:
import pandas as pd
import numpy as np

import duckdb

In [2]:
from src.datasets import daocensus_text, daocensus
# from src.datasets

In [3]:
ORG_NAME = 'Decentraland'

dfv, dfp = daocensus_text.get("./data/daos-census-text", ORG_NAME, 'snapshot')
dfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116560 entries, 0 to 116559
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   platform       116560 non-null  object        
 1   name           116560 non-null  object        
 2   id             116560 non-null  object        
 3   proposal       116560 non-null  category      
 4   deployment     116560 non-null  object        
 5   platform_vote  116560 non-null  object        
 6   voter          116560 non-null  category      
 7   date           116560 non-null  datetime64[ns]
 8   choice         116560 non-null  object        
 9   weight         116560 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(1), object(6)
memory usage: 7.9+ MB


In [4]:
dfv_old, dfp_old = daocensus.get("./data/daos-census", ORG_NAME, 'snapshot')

In [9]:
print(f'Len dfv_old: {len(dfv_old)}, dfv: {len(dfv)}')
print(f'Len dfp_old: {len(dfp_old)}, dfp: {len(dfp)}, {dfp["id"].nunique()}')

Len dfv_old: 116560, dfv: 116560
Len dfp_old: 1942, dfp: 1943, 1943


Hay propuestas nuevas, pero no tenemos los votos de esas propuestas

In [14]:
new_props = dfp[~dfp['id'].isin(dfp_old['id'])]
new_props

,platform,name,platform_deployment,id,deployment,platform_proposal,author,date,votes_count,title,description,start,end
233,snapshot,Decentraland,snapshot.dcl.eth,d2e7f401-7e51-56f5-84be-6819cdca8034,41fd8de5-f8e2-5023-86a3-825c49e9ad7f,Qme5zXJNfVSPzyiESQMnyx39MaJduX9MZEaTnmQ8Unj8ZR,0x5e23d08324f017d5425e59a2782c9ae27ace0958,2021-12-03 06:26:25,131,Change the price of claiming a unique name,> by 0xe83fcc89fb8fdd63eb542be01420d24257f28bc...,2021-12-03 06:26:00,2021-12-10 06:26:00
